# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

План исследования:  
1. Загрузить и подготовить данные.
2. Решить задачу умножения матриц
3. Разработать алгоритм преобразования данных (шифрование)
4. Проверить алгоритм
- На исходных данных
- На преобразованных данных
- Сравнить результаты
5. Общий вывод 

## Загрузка данных

In [1]:
!pip install ydata_profiling --user

In [2]:
import pandas as pd 
import numpy as np 
from ydata_profiling import ProfileReport

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
rs = 112263 

In [3]:
try:
    data = pd.read_csv('insurance.csv')    
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [4]:
ProfileReport(data)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Опираясь на отчет по данным, не стоит спешить с выводом, что есть дупликаты и пропуски. Данные обезличены - нет id или ФИО, поэтому совпадение по остальным столбцам можно считать случайностью. `zeros` стоит расценивать как отсутствие того или инного признака, в нашем случае - это отсутствие членов семьи - одиночки, и наличие страховых выплат, а точнее их отсутствие

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** R2 метрика не изменится 

**Обоснование:** далее:

свойства матричных операций
$$
(AB)^T = B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$

Домножив матрицу X на матрицу P шифрования, получаем
$$
w_n = ((XP)^T(XP))^{-1} (XP)^T y
$$

раскрыв скобки, получаем
$$
w_n = (P^T X^T XP)^{-1} P^TX^T y
$$

далее применив второе свойство матричных операций, мы получаем
$$
w_n = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

При условии, что $(P^T)^{-1} P^T = E$

уравнение имеет вид
$$
w_n = P^{-1} X^{-1} (X^T)^{-1} X^T y
$$
$$
w_n = P^{-1} (X^T X)^{-1} X^T y
$$

используя формулу обучения для свертки формулы, получаем 
$$
w_n = P^{-1} w
$$

таким образом, если $w_n$ заменить на $P^{-1} w$, мы прийдем к той же самой поставленной задаче,  
подставив в формулу предсказаний $a = X_w$ матрицу $X_n = XP$ с признаками $w_n$

$$
a_n = XP w_n a_n = XP P^{-1} w a_n = X w = a
$$

## Алгоритм преобразования

**Алгоритм**

Для шифрования данных будем использовать простой алгоритм линейного преобразования:

Проверить матрицу на обратимость: и если она обратима, переходим к следующему пункту, если нет, продолжаем генерировать, используя np.random, пока не добьемся обратимости

Оригинальный массив данных X перемножаем на сгерерированную случайную квадратную обратимую матрицу P

Обучение линейной регрессии на исходных и на зашифрованных данных.
Сравнение получившихся метрик R2

**Обоснование**

Такое преобразование не должно повлиять на качество метрики предсказания. 
Признаки хоть и изменены, но задача и значения целевого признака остались исходными, в формуле обучения теперь дополнительно фигурирует ранеупомянутая квадратная обратимая матрица
$w' = \arg\min_{w'} MSE(X' w', y)$

$w_n = ((XP)^{T} XP)^{-1} (XP)^{T} y$


$a' = X' w' = XP ((XP)^{T} XP)^{-1} (XP)^{T} y$

Такое преобразование необходимо произвести с незашифрованной выборкой, после чего сравнть полученные результаты R2 метрик обучения моделей: они должны быть одинаковы

Восстановить зашифрованные данные таким способом так же легко: достаточно перемножить преобразованную матрицу на обратную матрицу, однако такой задачи перед нами не стоит

$X' P^{-1} = XP P^{-1} = XE = X$

***Это простой метод шифрования данных, имеющий ряд недостатков, однако позволяет скрыть информацию от первичного просмотра и анализа для неподготовленного пользователя, к кому попали данные***

## Проверка алгоритма

### Подготовка данных для проверки

In [5]:
# Подготовка признаков

features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']
 
# Разделение данных на обучающую и валид выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=rs)

# Размеры выборок
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)

(3750, 4) (3750,)
(1250, 4) (1250,)


In [6]:
# Создание случайной обратимой матрицы через проверку на обратимость

def generate_matrix(matrix_size):
    det = 0
    n = matrix_size

    while det == 0:
        random_matrix = np.random.randint(10, size=(n, n))
        det = np.linalg.det(random_matrix)
    return random_matrix

In [7]:
n = features_train.shape[1]
random_matrix = generate_matrix(n)
random_matrix

array([[2, 9, 0, 9],
       [9, 6, 7, 1],
       [6, 1, 1, 2],
       [7, 8, 7, 1]])

### Модель линейной регрессии на исходных данных

In [8]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
r2_lr = r2_score(target_valid, predictions)
print(f'R2 модели на исходных данных: {r2_lr}')

R2 модели на исходных данных: 0.4249920284593577


### Модель линейной регрессии на преобразованных данных

Преобразуем признаки

In [9]:
features_rm = features @ random_matrix
features_rm.columns = features.columns
features_rm

,Пол,Возраст,Зарплата,Члены семьи
0,297978.0,49863.0,49894.0,99251.0
1,228421.0,38284.0,38329.0,76047.0
2,126261.0,21174.0,21203.0,42029.0
3,250403.0,41842.0,41861.0,83423.0
4,156854.0,26277.0,26296.0,52237.0
...,...,...,...,...
4995,214466.0,35884.0,35910.0,71430.0
4996,314713.0,52612.0,52645.0,104835.0
4997,203594.0,34036.0,34054.0,67822.0
4998,196421.0,32865.0,32875.0,65434.0


In [10]:
# Подготовка признаков преобразованных данных
 
# Разделение данных на обучающую и валид выборки
features_train_rm, features_valid_rm, target_train_rm, target_valid_rm = train_test_split(
    features_rm, target, test_size=0.25, random_state=rs)

# Размеры выборок
print(features_train_rm.shape, target_train_rm.shape)
print(features_valid_rm.shape, target_valid_rm.shape)

(3750, 4) (3750,)
(1250, 4) (1250,)


In [11]:
model = LinearRegression()
model.fit(features_train_rm, target_train_rm)
predictions_rm = model.predict(features_valid_rm)
r2_rm_lr = r2_score(target_valid_rm, predictions_rm)
print(f'R2 модели на зашифрованных данных: {r2_rm_lr}')

R2 модели на зашифрованных данных: 0.4249920284593717


### Сравнение результатов

In [12]:
print(r2_lr==r2_rm_lr)
print(f'R2 модели на исходных данных:        {r2_lr}')
print(f'R2 модели на зашифрованных данных:   {r2_rm_lr}')

False
R2 модели на исходных данных:        0.4249920284593577
R2 модели на зашифрованных данных:   0.4249920284593717


***Вывод по разделу***

Шифрование данных домножением признаков практически не повлияло на качество модели. R2 метрика хоть и отличается, но их разность настолько мала, что ее можно считать незначительной погрешностью в вычислениях


P.S:
Похожий метод скорее всего был применен к данным по проекту по разработке скважин, где были предоставлены данные `f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы)`, что позволило нам проводить анализ на преобразованных данных без критичных отклонений разности метрик

## Вывод

В рамках проекта по шифрованию данных клиентов компании был проведет первичный просмотр-анализ данных, предобработка по которому не потребовалась: пропуски и явные и неявные дупликаты отсутствуют. 

Был разработан и применен алгоритм преобразования (шифрования) данных таким образом, чтобы по ним было сложно восстановить персональную информацию. Приведено обоснование решения задачи. 

Алгоритм проверен моделью обучения, а мерой качества оценки послужила метрика R2 на модели Линейной регрессии. 

Модель машинного обучения была обучена на непреобразованных и преобарзованных данных, результаты оценки модели практически не отличаются, показатель отклонения на уровне незначительной погрешности вычислений. 